In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl sentencepiece triton

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [4]:
def format_prompt(ex):
  chat = ex["messages"]
  prompt = tokenizer.apply_chat_template(chat, tokenize=False)
  return {"text": prompt}

In [5]:
dataset = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft").shuffle(seed=42).select(range(3000))

In [6]:
dataset = dataset.map(format_prompt)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [7]:
print(dataset["text"][16])

<|user|>
Develop a 30-minute long podcast episode that discusses the various ways technology has influenced the processes, practices, and possibilities in the music industry. Include interviews with musicians, producers, and experts in the field to gain diverse perspectives on this topic. Explore specific examples of how technology has changed the music industry, including the rise of streaming services, the impact of social media, and the evolution of music production tools. Incorporate examples of both positive and negative effects of technology on music consumption, creative production, distribution, and revenue streams. Use a storytelling approach to engage listeners in understanding the complex relationship between technology and the music industry, and offer insights into future directions for this dynamic field.</s>
<|assistant|>
Introduction:

Music has always been an integral part of human culture, and with the advent of technology, its reach and influence have only grown. Tod

In [8]:
!pip install --upgrade bitsandbytes

In [9]:
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = "float16",
    bnb_4bit_use_double_quant = True
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [11]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

In [13]:

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

In [14]:
peft_config = LoraConfig(
    lora_alpha= 32,
    lora_dropout = 0.1,
    r = 64,
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules = ["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj", "down_proj"]
)

In [15]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [16]:
from transformers import TrainingArguments, DataCollatorForSeq2Seq

In [17]:
output_dir = "./results"
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)

In [18]:
from trl import SFTTrainer, SFTConfig

In [20]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=512,
    peft_config=peft_config,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:477: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [21]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abhinavm16104 (abhinavm16104-delhi-skill-and-entrepreneurship-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.671300
20,1.475600
30,1.450900
40,1.488400
50,1.477800
60,1.390500
70,1.494900
80,1.450000
90,1.427400
100,1.404000


TrainOutput(global_step=375, training_loss=1.4168808301289877, metrics={'train_runtime': 1462.0005, 'train_samples_per_second': 2.052, 'train_steps_per_second': 0.256, 'total_flos': 9994755938844672.0, 'train_loss': 1.4168808301289877, 'epoch': 1.0})

In [23]:
trainer.model.save_pretrained("TinyLlama-1.1B-qlora-mango")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
from peft import AutoPeftModelForCausalLM

In [24]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora-mango",
    low_cpu_mem_usage=True,
    device_map="auto"
)

In [25]:
merged_model = model.merge_and_unload()

In [26]:
from transformers import pipeline

In [27]:
prompt = """<|user|>
Tell me something about mangoes.</s>
<|assistant|>"""

In [28]:
pipe = pipeline(task="text-generation", model = merged_model, tokenizer = tokenizer)

In [29]:
print(pipe(prompt)[0]["generated_text"])

<|user|>
Tell me something about mangoes.</s>
<|assistant|>
Mangoes are a type of fruit that originated in Southeast Asia and are now grown in many parts of the world. They are known for their sweet, juicy, and tart flavor, and they are often used in desserts, salads, and other dishes. Mangoes are also a source of vitamin C, which is important for maintaining healthy skin, teeth, and bones.


In [ ]:
merged_model.save_pretrained("TinyLlama-1.1B-qlora-mango_model")

In [ ]:
tokenizer.save_pretrained("TinyLlama-1.1B-qlora-mango_model")